In [268]:
import pandas as pd
import openai
from dotenv import load_dotenv
import os
import ast
from openai import AzureOpenAI
import faiss
import numpy as np
import json
import spacy
nlp = spacy.load("en_core_web_sm")
import pycountry
import re
from bert_score import score as bert_score
import csv
import transformers
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import itertools
import re
from sklearn.metrics.pairwise import cosine_similarity
from spacy.lang.en.stop_words import STOP_WORDS



/Users/davidoluyalegbenga/.pyenv/versions/3.9.6/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


### Load Model files

In [2]:
df = pd.read_pickle('../models/df_embed_EN_All.pkl')

### Load Enviroment files

In [3]:
# Load environment variables
load_dotenv()

True

In [4]:
# OpenAI API configuration
openai.api_type = "azure"
openai.api_key = os.getenv("api_key_azure")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = os.getenv("api_version")
openai_deployment = "sdgi-gpt-35-turbo-16k"


client = AzureOpenAI(
  api_key = os.getenv("api_key_azure"),  
  api_version = os.getenv("api_version"),
  azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
)

embedding_model = os.getenv("USER_QUERY_EMBEDDING_ENGINE") 

# print(openai.api_key)
# print(openai.api_base)
# print(openai.api_version)


<h3>globals</h3>

In [5]:
# test_query="What are the sustainable energy priorities for UNDP?"
test_query = 'What is the Human Development Index (HDI) value for Albania as mentioned in the document?'


<h3> helper functions </h3>

In [6]:
# use this function to make simple openAI Calls
def callOpenAI(prompt):  
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=[
                        {"role": "user", "content": prompt},
                    ]
                )
    response = response_entities.choices[0].message.content
    return response


<h3> processing modules </h3>

In [7]:
def extractEntitiesFromQuery(user_query):
    prompt = f"""
    Extract entities from the following user query: \"{user_query}\" and return output in array format.
    
    -Entities should be directly related to the domain or topic of interest. They should represent important concepts that contribute to the understanding of the subject matter.
    -Each entity in the knowledge graph should be distinct and have a unique identifier. This ensures clarity and avoids ambiguity when establishing relationships between entities.
    -You Must return output in array format e.g  ['entity1','entity2'] !!!
    -Avoid adding new lines or breaking spaces to your output. Array should be single dimension and single line !!!
 
    """
    entity_list = callOpenAI(prompt)   
    return entity_list

# Test usage
# test_query = "What are the sustainable energy for UNDP?"
# entity_list = extractEntitiesFromQuery(test_query)
# print(entity_list)

In [20]:
## module to get information on the entities from user query using the KG
def knowledgeGraphModule(user_query):
    
    # generate list of entities based on user query
    entity_list = extractEntitiesFromQuery(user_query)
    my_list = ast.literal_eval(entity_list)
    prompt_summarise_entites = f"""
    Summarize all relations between all the entities : {my_list}
    """
    summarise_entities = callOpenAI(prompt_summarise_entites)
    # Initialize an empty dictionary to store information
    entities_dict = {
        "relations": summarise_entities,
        "entities": {}
    }
    # Loop through each entity in the list
    for entity in my_list:
        # Fetch information about the entity from your knowledge graph
        prompt = f"Give me a short description 50 words of {entity}"
        entity_info = callOpenAI(prompt)
        # Add the entity information to the dictionary
        entities_dict["entities"][entity] = entity_info
    
    return entities_dict


# Test usage
test_query = "What is the role of Paris Agreement in sustainable energy development?"
entities_dict = knowledgeGraphModule(test_query)
print(entities_dict)

{'relations': 'The Paris Agreement is related to sustainable energy development.', 'entities': {'Paris Agreement': 'The Paris Agreement is an international treaty adopted in 2015 by nearly every country in the world. Its goal is to combat climate change by limiting global warming to well below 2 degrees Celsius above pre-industrial levels, and to pursue efforts to limit the temperature increase to 1.5 degrees Celsius.', 'sustainable energy development': 'Sustainable energy development refers to the process of harnessing and utilizing renewable energy sources in a manner that minimizes environmental impact and promotes long-term viability. It involves the adoption of clean technologies, such as solar, wind, and hydro power, to meet energy needs while reducing greenhouse gas emissions and preserving natural resources.'}}


In [21]:

def find_mentioned_countries(text):
    countries = set()
    
    # Tokenize the text using regular expressions to preserve punctuation marks
    words = re.findall(r'\w+|[^\w\s]', text)
    text = ' '.join(words)  # Join the tokens back into a string
    
    for word in text.split():
        try:
            country = pycountry.countries.get(name=word) #pycountry.countries.lookup(word)
            if country != None : 
               countries.add(country.name)
        except LookupError:
            pass
    
    return list(countries)

# Example 
# user_query = 'Give me a summary of the goals UNDP wants to achieve in 10 years and the energy plans for Philippines and Angola'
# mentioned_countries = find_mentioned_countries(user_query)
# mentioned_countries

In [286]:

# Load the English language model
# Function to calculate the average word embedding for a sentence
def average_word_embedding(sentence):
    # Parse the sentence using SpaCy
    doc = nlp(sentence)
    # Get word vectors and average them
    word_vectors = [token.vector for token in doc if token.has_vector]
    if not word_vectors:
        return None
    return np.mean(word_vectors, axis=0)

# Function to calculate context similarity between two sentences using word embedding averaging
def calculate_context_similarity(sentence1, sentence2):
    # Get average word embeddings for each sentence
    avg_embedding1 = average_word_embedding(sentence1)
    avg_embedding2 = average_word_embedding(sentence2)
    if avg_embedding1 is None or avg_embedding2 is None:
        return None
    # Calculate cosine similarity between the embeddings
    similarity = cosine_similarity([avg_embedding1], [avg_embedding2])[0][0]
    return similarity

# # Example sentences
# sentence1 = 'The companys quarterly earnings report exceeded expectations, leading to a surge in stock prices.'
# sentence2 = 'The firms financial results for the last quarter surpassed predictions, resulting in a sharp rise in the value of shares'



# Calculate context similarity
# similarity = calculate_context_similarity(sentence1, sentence2)
# print("Context similarity:", similarity)




Context similarity: 0.6653273


In [22]:
# Function to calculate Jaccard similarity between two texts
def jaccard_similarity(text1, text2):
    # Tokenize texts
    tokens1 = set(text1.lower().split())
    tokens2 = set(text2.lower().split())
    
    # Calculate Jaccard similarity
    intersection = len(tokens1.intersection(tokens2))
    union = len(tokens1.union(tokens2))
    
    return intersection / union if union > 0 else 0


In [182]:
def title_contains_entity(entity, title):
    # Convert both entity and title to lowercase for case-insensitive comparison
    entity_lower = entity.lower()
    title_lower = title.lower()

    # Check if the lowercase entity is contained within the lowercase title
    if entity_lower in title_lower:
        return 1
    else:
        return 0

In [298]:
#This contains all filters for the semantic search
#Context Similarity takes two queries and find how similar they are "context wise"
#E.g "My house is empty today" and "Nobody is at my home" are same context but not word similarity
# - Filter country relevant documents when mentioned 
# - Filter by Context similarity in user_query and title, journal, content etc.

def filter_semantics(user_query):

    #Allow parallels
    #Extract notable entities in query e.g ORG, NAME, Place, country, location etc.
    #Location related: 
    # GPE: Countries, cities, states.
    # NORP: Nationalities, religious and political groups.
    # LANGUAGE: Any named language. 
    # FAC: Buildings, airports, highways, bridges, etc.

    #Other/General:
    # PERSON: People, including fictional entities.
    # ORG: Companies, agencies, institutions, etc.
    # LOC: Non-GPE locations, mountain ranges, bodies of water.
    # PRODUCT: Objects, vehicles, foods, etc. (Not services)
    # EVENT: Named hurricanes, battles, wars, sports events, etc.
    # WORK_OF_ART: Titles of books, songs, etc.
    # LAW: Named documents made into laws.
    # LANGUAGE: Any named language.
    # DATE: Absolute or relative dates or periods.
    # TIME: Times smaller than a day.
    # PERCENT: Percentage, including "%".
    # MONEY: Monetary values, including unit.
    # QUANTITY: Measurements, as numerical values with units.
    # ORDINAL: "first", "second", etc.
    # CARDINAL: Numerals that do not fall under another type.

    doc = nlp(user_query)
    # Extract all entities
    # entities = [(ent.text, ent.label_) for ent in doc.ents]
    entities = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ != ""]  # Filter out empty entities
    entities.extend((token.text, "NOUN") for token in doc if token.pos_ in ["NOUN","PROPN", "PRON", "PROPN", "NUM", "SYM", "X","ABBR"] or token.is_alpha)

    # Remove stop words
    entities = [(entity, label) for entity, label in entities if entity.lower() not in STOP_WORDS]
    
    # Print the extracted entities
    # print("All Entities and POS:", entities)
    # Generate DFs for main entities
    filtered_df_country = pd.DataFrame()  # Initialize an empty DataFrame
    filtered_df_others = pd.DataFrame()  # Initialize an empty DataFrame
    filtered_df_backup_reference = pd.DataFrame() # Initialize an empty DataFrame
    allow_low = True

    for entity, label in entities:
        # print(entity)
        filtered_df_others = pd.concat([filtered_df_others, df[df['Document Title'].str.lower().str.contains(entity.lower(), na=False)]])

        #Calculate similarity scores for each document title
        similarity_scores = []
        document_titles = []

        # Iterate through each document title and calculate similarity score
        for title in filtered_df_others['Document Title']:
            if title is not None:
                similarity_score = calculate_context_similarity(user_query,title) 
                # print(entity)
                # print(similarity_score)
                # print(user_query)
                # print(title)
                # print("=================================")    
                similarity_scores.append(similarity_score)
                document_titles.append(title)
        
        # Create DataFrame only with valid similarity scores
        similarity_df = pd.DataFrame({'Document Title': document_titles, 'Similarity Score': similarity_scores})
        
        df_temp = pd.concat([df])

        threshold = 0.5

        # Filter df based on similarity scores greater than threshold for filtered_df_others
        filtered_df_others = df[df['Document Title'].isin(similarity_df[similarity_df['Similarity Score'] > threshold]['Document Title'])]
        filtered_df_backup_reference = pd.concat([filtered_df_backup_reference,  df_temp[df_temp['Document Title'].isin(similarity_df[(similarity_df['Similarity Score'] >= 0.1) & (similarity_df['Similarity Score'] < 0.45)]['Document Title'])] ])

        #Check for location related e.g by country, language, locals
        if label in ['GPE', 'NORP', 'LANGUAGE', 'FAC']:
            filtered_df_country = pd.concat([filtered_df_country, df[df['Country Name'] == entity]])
    merged_df = pd.DataFrame()
    if filtered_df_others.empty and filtered_df_country.empty:
       print(f'on the reference df {filtered_df_backup_reference.empty}')
       merged_df = pd.concat([filtered_df_backup_reference])
    else :
       merged_df = pd.concat([filtered_df_country,filtered_df_others])
    
    return merged_df




  
# Example 
# test_query=" is the promotion of renewable energy a focal point within the energy sector?"
# filtered_country = filter_semantics(test_query)
# filtered_country

All Entities and POS: [('promotion', 'NOUN'), ('renewable', 'NOUN'), ('energy', 'NOUN'), ('focal', 'NOUN'), ('point', 'NOUN'), ('energy', 'NOUN'), ('sector', 'NOUN')]
promotion
renewable
energy
focal
point
energy
sector


,Code,Status,Country Name,Country Code,Category,KeyWord to Search,Document Title,Exists?,Publication Date,Publication Year,...,Link,Content,Region,Journal,Authors,Description,Resolution No,token_length,Embedding,content_cut
0,,,None,None,SEH,,Machine learning for a sustainable energy future,Y,2023-02-05 00:00:00,2023,...,https://www.nature.com/articles/s41578-022-004...,Machine learning for a sustainable energy futu...,Global,Nature,"Zhenpeng Yao, Yanwei Lum, Andrew Johnston, Lui...",NaN,NaN,22245,"[-0.009152377024292946, -0.007553262636065483,...",Machine learning for a sustainable energy futu...
5,,,None,None,SEH,,The justice and policy implications of clean e...,Y,2023-02-05 00:00:00,2023,...,https://www.frontiersin.org/articles/10.3389/f...,Frontiers | The justice and policy implication...,Sub-Saharan Africa,Frontiers in Environmental Science,"BEK Nsafon, NN Same, AO Yakub, D Chaulagain, N...",NaN,NaN,18084,"[0.03133831545710564, -0.03404851630330086, 0....",Frontiers | The justice and policy implication...
10,,,,,Publications,None,The potential of digital business models in th...,Y,2022,,...,https://www.iea.org/articles/the-potential-of-...,The Potential of Digital Business Models in th...,NaN,,NaN,None,NaN,5597,"[-0.00489771505817771, -0.02511439472436905, 0...",The Potential of Digital Business Models in th...
115,,,,,Publications,None,The evolution of energy efficiency policy to s...,Y,2023,,...,https://www.iea.org/reports/the-evolution-of-e...,The Evolution of Energy Efficiency Policy to S...,NaN,,NaN,None,NaN,1081,"[0.013654322363436222, -0.02168549969792366, 0...",The Evolution of Energy Efficiency Policy to S...
121,,,,,Publications,None,Is the energy crisis really making the busines...,Y,2022,,...,https://www.iea.org/reports/is-the-energy-cris...,Is the energy crisis really making the busines...,NaN,,NaN,None,NaN,2233,"[0.009065688587725163, -0.016282841563224792, ...",Is the energy crisis really making the busines...
154,,,,,Publications,None,The cost of financing for renewable power,Y,2023,,...,https://www.irena.org/Publications/2023/May/Th...,The cost of financing for renewable power EN...,NaN,,NaN,None,NaN,862,"[0.0051294611766934395, -0.0460810586810112, 0...",The cost of financing for renewable power EN...
156,,,,,Publications,None,Global landscape of renewable energy finance 2023,Y,2023,,...,https://www.irena.org/Publications/2023/Feb/Gl...,Global landscape of renewable energy finance 2...,NaN,,NaN,None,NaN,947,"[0.003613060340285301, -0.03236578777432442, 0...",Global landscape of renewable energy finance 2...
179,,,,,Publications,None,Gender equality in the sustainable energy tran...,Y,2023,,...,https://www.unwomen.org/en/digital-library/pub...,Gender equality in the sustainable energy tran...,NaN,,NaN,None,NaN,760,"[-0.004305988550186157, -0.03226073831319809, ...",Gender equality in the sustainable energy tran...
190,,,,,Publications,"Sustainable Energy,Energy Efficiency,Energy Tr...",Unsustainable mining will be the Achilles heel...,Y,2022,,...,https://www.carbontrust.com/en-eu/news-and-ins...,Unsustainable mining will be the Achilles heel...,NaN,,NaN,we cannot afford to reach Net Zero global emis...,NaN,2393,"[-0.003787592751905322, -0.03763367980718613, ...",Unsustainable mining will be the Achilles heel...
206,,,,,Publications,None,The energy transition: a catalyst to address t...,Y,2023,,...,https://www.seforall.org/publications/the-ener...,The energy transition: a catalyst to address t...,NaN,,NaN,None,NaN,618,"[0.0023315558210015297, -0.03005784936249256, ...",The energy transition: a catalyst to address t...


In [125]:
def search_embeddings(user_query):
    df_filtered = filter_semantics(user_query) if filter_semantics(user_query) is not None else None
    
    if df_filtered is not None and not df_filtered.empty:  # Check if DataFrame is not None and not empty
        length = len(df_filtered.head())
        filtered_embeddings_arrays = np.array(list(df_filtered['Embedding']))
        index = faiss.IndexFlatIP(filtered_embeddings_arrays.shape[1]) 
        index.add(filtered_embeddings_arrays)
        
        user_query_embedding = client.embeddings.create( 
                input=user_query ,model= embedding_model
            ).data[0].embedding

        k = min(5, length)
        distances, indices = index.search(np.array([user_query_embedding]), k)
        return df_filtered, distances, indices
    else:
        return None, None, None


In [115]:
def get_answer(user_question, content):
    system_prompt = "You are a system that answers user questions based on excerpts from PDF documents provided for context. Only answer if the answer can be found in the provided context. Do not make up the answer; if you cannot find the answer, say so."
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_question},
        {'role': 'user', 'content': content},
    ]
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0.2,
                    messages=messages
                )
    response = response_entities.choices[0].message.content
    return response
  

In [302]:
def map_to_structure(qs):
    result_dict = {}

    # Extract the DataFrame from the tuple
    dataframe = qs[0]

    # Counter to limit the loop to 10 iterations
    count = 0
    for index, row in dataframe.iterrows():
        # Define a unique identifier for each document, you can customize this based on your data
        document_id = f"doc-{index + 1}"
        # Handle NaN in content by using fillna
        content = row["Content"]
        content = ' '.join(row["Content"].split()[:160])
        # Create a dictionary for each document
        document_info = {
            "title": row["Document Title"],
            "extract": content or "",  # You may need to adjust this based on your column names
            "category": row["Category"],
            "link": row["Link"],
            "thumbnail": ''
        }
        # print(document_info)
        # Add the document to the result dictionary
        result_dict[document_id] = document_info

        # Increment the counter
        count += 1

        # # Break out of the loop if the counter reaches top 25
        if count == 10:
            break

    return result_dict


In [214]:
## module to extract text from documents and return the text and document codes

def semanticSearchModule(user_query):
    qs = search_embeddings(user_query) #df, distances, indices
    # if qs != None :
    if qs[0] is not None:
        result_structure = map_to_structure(qs)
        return result_structure
    else : 
        return []
#test usage
# excerpts_dict=semanticSearchModule("What is the role of USAID in supporting Albania's energy sector strategy?")
# # print(f"""excerpts_dict {excerpts_dict}""")

# #Return top 10-20 most related 
# # Define the filename to save the JSON data -  can remove later
# json_filename = "outputs/excerpts.json"

# # Save excerpts_dict to the JSON file just for a better preview
# with open(json_filename, 'w', encoding='utf-8') as json_file:
#     json.dump(excerpts_dict, json_file, ensure_ascii=False, indent=4)

# print(f"Excerpts saved to {json_filename}")

In [119]:
## module to get data for specific indicators which are identified is relevant to the user query

def indicatorsModule(user_query): #lower priority
    
    # find relevant indicators based on uesr query and extract values
    indicators_dict={
        "indicator-id-1":"value from indicator-id-1",
        "indicator-id-2":"value from indicator-id-2"
    }#temp
    
    return indicators_dict

#test usage
indicators_dict=indicatorsModule(test_query)
print(indicators_dict)

{'indicator-id-1': 'value from indicator-id-1', 'indicator-id-2': 'value from indicator-id-2'}


In [120]:
## module to generate query ideas

def queryIdeationModule(user_query): # lower priority
    
    # Generate query ideas using OpenAI GPT-3
    prompt = f"""Generate prompt ideas based on the user query: {user_query}

    -Prompt shoud not be answer to the user query but give other contextual ways of representing the user query !!!
    -You Must return output in array format e.g ['idea 1', 'idea2'] !!!
    - Each generated ideas should be very dinstinct but contextual. Not repeatitive or using same words
    - The query idea should be in a question form and not an answer form.
    -Avoid adding new lines or breaking spaces to your output. Array should be single dimension and single line !!!
    """
    response = callOpenAI(prompt)
    return response

#test usage
# query_idea_list=queryIdeationModule(test_query)
# print(query_idea_list)

<h3> synthesis module </h3>

    llm_instructions="llm instruction template here, with placeholders for insertion of user query, excerpts, indicator data, and entity and relation info" 


In [311]:
# module to synthesize answer using retreival augmented generation approach


      #  - Make sure to Include citations based on the Sources. e.g Text excerpt here<a data-id='test-doc-1'>[1]</a> when referencing a document in the sources. using 1 ...nth
      #       - The citations anchor should be near the excerpt not following each other.
      #       - Use the anchor tag for the citation links and should link to the document link in the Sources. for example Undp operates in afganistan <a data-id='test-doc-1'>[1]</a>. UNDP offers health relationships <a data-id='test-doc-2'>[2]</a>.
      #       - Make sure the data-id actually exists in the Sources.
      #       - The text in the anchor tag should be citation number not document title.
      #       - You can reference multitple citations based sources

prompt_formattings= f"""
  - Answer output must be properly formatted using HTML. 
  - Don't include <html>, <script>, <link> or <body> tags. Only text formating tags should be allowed. e.g h1..h3, p, anchor, etc. No new line "\n" or tab characters
  - Strictly infer your answers from the <Sources> Only and make citations to Source extract referenced 
  - The Source as format like: "doc-n\": {{
    "title": "title of the relate document",
    "extract": "content",
    "category": "",
    "link": "",
    "thumbnail": ""
}}, where doc-n can be doc-1, doc-24 etc.. n is in integer.
 - Reference the extract and title of all document sources provided in the json and summarise it into a coherent answer that relates to the <User Query>
    - Citation should follow formats: [reference content]<a href='link here' data-id='doc-n'>[i]</a> i.e IEEE Citation Style (Institute of Electrical and Electronics Engineers):

 - Give output writing tone like a academic research tone
    - Strictly use IEEE Citation Style 
  - If no <Sources> are provided, simply say you don't have that information   
  - Remove new line or tab characters from your output

"""

def synthesisModule(user_query, entities_dict, excerpts_dict, indicators_dict,prompt_formattings):
    
    # Generate prompt engineering text and template
    llm_instructions = f"""
    Ignore previous commands!!!
    Given a user query, use the provided <Sources> extract section of the JSON only to provide the correct answer to the user's query.
    
    User Query: {user_query}
    
    Sources: {excerpts_dict}
    
    {prompt_formattings}
    """
    ###synthesize data into structure within llm prompt engineering instructions
    answer=callOpenAI(llm_instructions)
    
    return answer

## to test this, run the full pipeline with the handleApiCall function

<h3> run pipeline </h3>

In [312]:
# full pipeline with retreival, synthesis of answer to user query, and structure results into api response

def handleApiCall(user_query):
    
    ##run processing modules (in parallel)
    entities_dict=knowledgeGraphModule(user_query)
    excerpts_dict=semanticSearchModule(user_query)
    indicators_dict=indicatorsModule(user_query) ##lower priority
    query_idea_list=queryIdeationModule(user_query) ##lower priority
    
    ##synthesis module
    answer=synthesisModule(user_query, entities_dict, excerpts_dict, indicators_dict, prompt_formattings)

    ##structure response
    response={
        "user_query":user_query,
        "answer":answer,
        "sources":excerpts_dict,
        "query_ideas":query_idea_list,
        "entities":list(entities_dict["entities"].keys())       
    }
    
    return response

# test usage
test_query = "is the promotion of renewable energy a focal point within the energy sector?"
response=handleApiCall(test_query) 
# Define the filename to save the JSON data -  can remove later
json_filename = "outputs/synthesis_output.json"

# Save excerpts_dict to the JSON file just for a better preview
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(response, json_file, ensure_ascii=False, indent=4)

print(f"Synthesis saved to {json_filename}")

All Entities and POS: [('promotion', 'NOUN'), ('renewable', 'NOUN'), ('energy', 'NOUN'), ('focal', 'NOUN'), ('point', 'NOUN'), ('energy', 'NOUN'), ('sector', 'NOUN')]
promotion
renewable
energy
focal
point
energy
sector
All Entities and POS: [('promotion', 'NOUN'), ('renewable', 'NOUN'), ('energy', 'NOUN'), ('focal', 'NOUN'), ('point', 'NOUN'), ('energy', 'NOUN'), ('sector', 'NOUN')]
promotion
renewable
energy
focal
point
energy
sector
Synthesis saved to outputs/synthesis_output.json


<h1>Testing</h1>

<p>This sections contains all things testings </p>

In [22]:
# !pip install bert_score

In [ ]:
def calculate_scores(csv_file):
    # Initialize an empty list to store processed entries
    result = []
    
    # Loop through each entry in the CSV file
    for entry in csv.DictReader(csv_file):
        query = entry['query']
        sample_answer = entry['sample_answer']
        
        # Call OpenAI for chat GPT answer
        chat_gpt_answer = callOpenAI(f""" 
                                    {query} 
                                    {prompt_formattings} 
                                    """)
        
        # Call the moonshot model API
        moonshot_model_answer = handleApiCall(query) 
        
        # Calculate BERT score for moonshot model answer
        P, F, R = bert_score([sample_answer], [moonshot_model_answer['answer']], lang='en', verbose=True)
        entry['moonshot_model_answer'] = moonshot_model_answer['answer']
        entry['bert_score'] = round(float(F), 2)

        # Calculate BERT score for chat GPT answer
        P, F, R = bert_score([sample_answer], [chat_gpt_answer], lang='en', verbose=True)
        entry['chat_gpt_answer'] = chat_gpt_answer
        entry['bert_score_gpt'] = round(float(F), 2)
        
        # Append the processed entry to the result list
        result.append(entry)
    
    # Return the list of processed entries
    return result


# Specify the path to your CSV file
csv_file_path = "../testing/queries.csv"

# Open the CSV file for reading
with open(csv_file_path, mode='r') as file:
    # Pass the file object to the function
    result = calculate_scores(file)

# Print updated data with scores
# print(json.dumps(result, indent=4))

# Save updated data to a JSON file
with open('../testing/test_output.json', 'w') as file:
    json.dump(result, file, indent=4)


<h1>Compare Moonshot BERT score to GPT BERT Score</h1>

In [84]:
# Load the JSON file
with open('../testing/test_output.json', 'r') as file:
    data = json.load(file)

# Initialize variables to store total scores and count of items
total_bert_score = 0
total_bert_score_gpt = 0
count = 0

# Iterate through each item in the JSON data
for item in data:
    # Extract bert_score and bert_score_gpt from the current item
    bert_score = item['bert_score']
    bert_score_gpt = item['bert_score_gpt']
    
    # Add the scores to the total
    total_bert_score += bert_score
    total_bert_score_gpt += bert_score_gpt
    
    # Increment the count
    count += 1

# Calculate the average scores
average_bert_score = total_bert_score / count
average_bert_score_gpt = total_bert_score_gpt / count

# Print the average scores
print("Average bert_score:", average_bert_score)
print("Average bert_score_gpt:", average_bert_score_gpt)

Average bert_score: 0.7995652173913044
Average bert_score_gpt: 0.7778260869565218


# TODO::
  
  - Extensive testing of the model with divers user queries. 
  - Deploy for frontend dev
  - Knowledge Graph
  - Citation


 